In [1]:
%pip install Recommenders 

     |████████████████████████████████| 318 kB 4.0 MB/s 
     |████████████████████████████████| 197 kB 37.9 MB/s 
     |████████████████████████████████| 12.4 MB 32.7 MB/s 
     |████████████████████████████████| 61 kB 5.7 MB/s 
     |████████████████████████████████| 1.5 MB 46.7 MB/s 
     |████████████████████████████████| 22.3 MB 1.4 MB/s 
     |████████████████████████████████| 310 kB 49.0 MB/s 
     |████████████████████████████████| 93 kB 2.1 MB/s 
     |████████████████████████████████| 636 kB 40.8 MB/s 
     |████████████████████████████████| 3.8 MB 36.2 MB/s 
     |████████████████████████████████| 749 kB 48.3 MB/s 
     |████████████████████████████████| 10.9 MB 41.9 MB/s 
     |████████████████████████████████| 378 kB 50.7 MB/s 
     |████████████████████████████████| 6.5 MB 36.9 MB/s 
     |████████████████████████████████| 895 kB 50.0 MB/s 
     |████████████████████████████████| 67 kB 4.8 MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.wh

In [1]:
import zipfile
zipfile.ZipFile('reco_utils.zip').extractall() 

In [2]:
import sys
#sys.path.append("../../")
import time
import os
import shutil
# import papermill as pm
import pandas as pd
import numpy as np
%tensorflow_version 1.x

import tensorflow as tf
#print(tensorflow.__version__)
from reco_utils.recommender.ncf.ncf_singlenode import NCF
from reco_utils.recommender.ncf.dataset import Dataset as NCFDataset
#from reco_utils.dataset import movielens
from reco_utils.dataset.python_splitters import python_random_split

from reco_utils.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items)
from reco_utils.common.constants import SEED as DEFAULT_SEED

# top k items to recommend
TOP_K = 10

# Model parameters
EPOCHS = 100
BATCH_SIZE = 256

SEED = DEFAULT_SEED  # Set None for non-deterministic results


TensorFlow 1.x selected.


In [3]:
df = pd.read_csv("BX-Book-Ratings.csv", sep=";", header=0, encoding="ISO-8859-1")

In [4]:
df.columns=['userID','itemID','rating']

In [5]:
df.head()

,userID,itemID,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [6]:
df['rating']=pd.to_numeric(df['rating'])

In [8]:
from reco_utils.dataset.python_splitters import python_random_split

# 75%비중으로 나누기 
train, test = python_random_split(df, 0.75)

In [ ]:
# user id, item id ,rating 가 있는 데이터만 가져옴 
data = NCFDataset(train=train, test=test, seed=SEED)

tf.compat.v1.disable_eager_execution()

In [ ]:
df.head()

In [ ]:
model = NCF(n_users=data.n_users,n_items=data.n_items,model_type="NeuMF",n_factors=4,layer_sizes=[16,8,4],\
            n_epochs=EPOCHS,batch_size=BATCH_SIZE,learning_rate=1e-3,verbose=10,seed=SEED)
# n_factors : latent space의 차원 수 (사용자 설정)
# layer_sizes: MLP 레이어 숫자 

# 모델학습 
model.fit(data)

predictions = [[row.userID, row.itemID, model.predict(row.userID, row.itemID)]
               for (_, row) in test.iterrows()]

# 예측값 저장하기 
predictions = pd.DataFrame(predictions, columns=['userID', 'itemID', 'prediction'])
predictions.head()